# Analizar sentimientos en Twitter

En el siguiente ejercicio visualizaremos en base a una consulta la cantidad de tweets positivos, neutrales y negativos.
Utilizaremos las librerias
* Tweepy : libreria oficial de la API de Twitter
* TextBlob : Libreria para el procesamiento de texto

### Twepy
Se puede instalar con el siguiente comando en la terminal
            conda install -c conda-forge tweepy
### TextBlob
Se puede instalar con el sigueinte comando en la terminal
            conda install -c conda-forge textblob
para el correcto funcionamiento de TextBlob, ocupamos intalar NLTK con el siguiente comando en la terminal
            python -m textblob.download_corpora
### Autentificacion
Para obtener informacion de los tweets generados, requerimos utilizar la API de Twitter. Por tal motivo requerimos registrar una app.
* Abrir el siguiente [link](http://apps.twitter.com) y hacer click en el boton de "Crreate New App"
* Llena los campos correspondientes. Puedes dejar en blanco el espacio de callback url
* Una vez creada la aplicacion, se te redirigira a la pagina de la aplicacion
* Abre la pestania 'Keys and Access Tokens'
* Copia 'Consumer Key', 'Consumer Secret', 'Access token' y 'Access Token Secret'

In [1]:
# librerias
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob

In [5]:
class TwitterClient(object):
    '''
    Generic Twitter Class for sentiment analysis.
    '''
    def __init__(self):
        '''
        Class constructor or initialization method.
        '''
        # keys and tokens from the Twitter Dev Console
        consumer_key = 'xxxxxxxxxx'
        consumer_secret = 'xxxxxxxxxx'
        access_token = 'xxxxxxxxxx'
        access_token_secret = 'xxxxxxxxxx'
 
        # attempt authentication
        try:
            # create OAuthHandler object
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            # set access token and secret
            self.auth.set_access_token(access_token, access_token_secret)
            # create tweepy API object to fetch tweets
            self.api = tweepy.API(self.auth)
        except:
            print("Error: Authentication Failed")
 
    def clean_tweet(self, tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])(\w+:\/\/\S+)", " ", tweet).split())
 
    def get_tweet_sentiment(self, tweet):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(self.clean_tweet(tweet))
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'
 
    def get_tweets(self, query, count = 10):
        '''
        Main function to fetch tweets and parse them.
        '''
        # empty list to store parsed tweets
        tweets = []
 
        try:
            # call twitter api to fetch tweets
            fetched_tweets = self.api.search(q = query, count = count)
 
            # parsing tweets one by one
            for tweet in fetched_tweets:
                # empty dictionary to store required params of a tweet
                parsed_tweet = {}
 
                # saving text of tweet
                parsed_tweet['text'] = tweet.text
                # saving sentiment of tweet
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)
 
                # appending parsed tweet to tweets list
                if tweet.retweet_count > 0:
                    # if tweet has retweets, ensure that it is appended only once
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)
 
            # return parsed tweets
            return tweets
 
        except tweepy.TweepError as e:
            # print error (if any)
            print("Error : " + str(e))

In [12]:
def main():
    # creating object of TwitterClient Class
    api = TwitterClient()
    # calling function to get tweets
    tweets = api.get_tweets(query = '#INIFAP', count = 200)
 
    # picking positive tweets from tweets
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
    # percentage of positive tweets
    print("Positive tweets percentage: {} %".format((len(ptweets)/len(tweets))*100))
    # picking negative tweets from tweets
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
    # percentage of negative tweets
    print("Negative tweets percentage: {} %".format((len(ntweets)/len(tweets))*100))
    # percentage of neutral tweets
    print("Neutral tweets percentage: {} % \ ".format((len(tweets) - len(ntweets) - len(ptweets))/len(tweets)*100))
 
    # printing first 5 positive tweets
    print("\n\nPositive tweets:")
    for tweet in ptweets[:10]:
        print(tweet['text'])
 
    # printing first 5 negative tweets
    print("\n\nNegative tweets:")
    for tweet in ntweets[:10]:
        print(tweet['text'])

In [13]:
if __name__ == "__main__":
    # calling main function
    main()

Positive tweets percentage: 3.571428571428571 %
Negative tweets percentage: 0.0 %
Neutral tweets percentage: 96.42857142857143 % \ 


Positive tweets:
Designa titular de la SAGARPA al Dr. Rafael Ambriz Cervantes como encargado del Despacho del #Ciencia #Inifap… https://t.co/Ublo8aVbbK


Negative tweets:
